<a href="https://colab.research.google.com/github/huyminh1115/Trip-Advisor-Hotel-Project/blob/main/FInal_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    gpu_name = tf.config.list_physical_devices('GPU')[0]
    print("GPU in use:", gpu_name)
else:
    print("GPU is not available.")


GPU is available.
GPU in use: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from wordcloud import WordCloud

# Preprocessing and evaluation
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_data.csv")

In [5]:
data.head()

,Review,Rating,words,word_count,tokenized,tokenized_words,new_rating,label
0,nice hotel expensive parking got good deal sta...,4,"['nice', 'hotel', 'expensive', 'parking', 'got...",87,nice hotel expensive parking get good deal sta...,"['nice', 'hotel', 'expensive', 'parking', 'get...",average,2
1,ok nothing special charge diamond member hilto...,2,"['ok', 'nothing', 'special', 'charge', 'diamon...",250,ok nothing special charge diamond member hilto...,"['ok', 'nothing', 'special', 'charge', 'diamon...",poor,0
2,nice rooms not 4* experience hotel monaco seat...,3,"['nice', 'rooms', 'not', '4*', 'experience', '...",217,nice room not experience hotel monaco seattle ...,"['nice', 'room', 'not', 'experience', 'hotel',...",average,1
3,"unique, great stay, wonderful time hotel monac...",5,"['unique,', 'great', 'stay,', 'wonderful', 'ti...",89,unique great stay wonderful time hotel monaco ...,"['unique', 'great', 'stay', 'wonderful', 'time...",excellent,2
4,"great stay great stay, went seahawk game aweso...",5,"['great', 'stay', 'great', 'stay,', 'went', 's...",191,great stay great stay go seahawk game awesome ...,"['great', 'stay', 'great', 'stay', 'go', 'seah...",excellent,2


In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np
import time

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize function for text data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Custom Dataset class for Hugging Face compatibility
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Initialize KFold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Store results for each fold
results = []

# Perform 10-fold cross-validation
for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data['tokenized']), start=1):
    print(f"\nRunning Fold {fold_idx}...")

    # Split data into train and test sets for this fold
    X_train, X_test = data['tokenized'][train_idx], data['tokenized'][test_idx]
    y_train, y_test = data['label'][train_idx], data['label'][test_idx]

    # Tokenize and create datasets
    train_encodings = tokenize_function(X_train.tolist())
    test_encodings = tokenize_function(X_test.tolist())

    train_dataset = CustomDataset(train_encodings, y_train.tolist())
    test_dataset = CustomDataset(test_encodings, y_test.tolist())

    # Initialize BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,  # Set small value initially with early stopping
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True
    )

    # Initialize Trainer with EarlyStopping
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    # Train the model
    start_time = time.time()
    trainer.train()
    elapsed_time = time.time() - start_time

    # Evaluate the model on the test set
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)

    # Calculate metrics
    accuracy = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds, average='weighted')
    precision = precision_score(y_test, preds, average='weighted')
    recall = recall_score(y_test, preds, average='weighted')

    # Store results for this fold
    results.append({
        "Fold": fold_idx,
        "Accuracy": accuracy,
        "F1": f1,
        "Precision": precision,
        "Recall": recall,
        "Time": elapsed_time
    })

    # Print fold results
    print(f"Fold {fold_idx} Results:")
    print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    print(f"  Time Taken: {elapsed_time:.2f} seconds\n")

# Calculate and display averaged metrics across all folds
avg_metrics = {
    metric: np.mean([result[metric] for result in results])
    for metric in ["Accuracy", "F1", "Precision", "Recall", "Time"]
}

print("\nAveraged Results Across All Folds:")
print(f"  Accuracy: {avg_metrics['Accuracy']:.4f}")
print(f"  F1 Score: {avg_metrics['F1']:.4f}")
print(f"  Precision: {avg_metrics['Precision']:.4f}")
print(f"  Recall: {avg_metrics['Recall']:.4f}")
print(f"  Average Time Taken per Fold: {avg_metrics['Time']:.2f} seconds")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,0.369600,0.343762


<ipython-input-6-d6613f0bf4a3>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.369600,0.343762
2,0.461300,0.381476


<ipython-input-6-d6613f0bf4a3>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Fold 10 Results:
  Accuracy: 0.8775, F1: 0.8711, Precision: 0.8668, Recall: 0.8775
  Time Taken: 990.98 seconds


Averaged Results Across All Folds:
  Accuracy: 0.8720
  F1 Score: 0.8595
  Precision: 0.8580
  Recall: 0.8720
  Average Time Taken per Fold: 1086.62 seconds
